**Proceso ETL**  

**Continuidad proceso de limpieza y calidad de datos**

Iniciamos cargando los archivos de vuelos y aeropuertos, y se contina con el proceso de perfilamiento y limpieza de datos iniciados en semanas previas.  

Revise un poco mas a fondo los datos de carga_ofrecida y carga a bordo, y esta vez opte por tratar la información de vuelos de tipo outlier donde tenemos carga_ofrecida < carga_bordo  

De igual manera se crearan las estructuras correspondientes a cada dimension y tabla de hecho.


In [9]:
from pyspark.sql import SparkSession
from pyspark.sql import functions
from pyspark.sql.types import StructType
from pyspark import SparkContext, SparkConf, SQLContext
from pyspark.sql.types import FloatType
from pyspark.sql.functions import udf

In [10]:
import os 
os.environ['PYSPARK_SUBMIT_ARGS'] = '--jars /usr/share/java/mariadb-java-client-2.5.3.jar pyspark-shell'

In [11]:
#Configuración de la sesión
spark_context = SparkContext()
sql_context = SQLContext(spark_context)
spark = sql_context.sparkSession
spark

In [4]:
PATH = "./data/"
df_vuelos = spark.read.load(PATH+"vuelos_tratado.csv",format="csv", sep=",", inferSchema="true", header="true")
df_vuelos.show()

df_aerop = spark.read.load(PATH+"aeropuertos_tratado.csv",format="csv", sep=",", inferSchema="true", header="true")
df_aerop.show()



+---+----+---+------+-------+-----------+----------+-------+--------------------+------+------+--------------+---------+------------------+-------+
|_c0| ano|mes|origen|destino|tipo_equipo|tipo_vuelo|trafico|             empresa|vuelos|sillas|carga_ofrecida|pasajeros|       carga_bordo|AnioMes|
+---+----+---+------+-------+-----------+----------+-------+--------------------+------+------+--------------+---------+------------------+-------+
|  0|2012|  1|   BOG|    CUC|       BE20|         T|      N|             RIO SUR|   1.0|   0.0|           0.0|      4.0|             100.0| 201201|
|  1|2013|  5|   UIB|    BOG|       DH8D|         R|      N|               AIRES|  30.0|1110.0|       24000.0|    873.0|            4222.0| 201305|
|  2|2013| 10|   IBE|    BOG|       DH8D|         R|      N|               AIRES|  98.0|3626.0|       56056.0|   2866.0|           2323.75| 201310|
|  3|2012|  4|   FLA|    BOG|       JS32|         T|      N|         SARPA S.A.S|   1.0|   0.0|           0.0|  

Se quitan duplicados de los vuelos en registros que son exactamente iguales.

In [5]:
print(df_vuelos.count())
df_vuelos=df_vuelos.drop('_c0').dropDuplicates()
print(df_vuelos.count())







67599
52234


Se identifican registros de vuelos con carga_bordo > carga_ofrecida, en estos casos, y se trata de dimensionar que tan grave es  o que tantos registros estan afectados por esto.

In [8]:

df_vuelos_outlier=df_vuelos.where(df_vuelos['carga_ofrecida']<df_vuelos['carga_bordo'])
df_vuelos_outlier.select('carga_ofrecida','carga_bordo').show()

+--------------+------------------+
|carga_ofrecida|       carga_bordo|
+--------------+------------------+
|           0.0|             180.0|
|           0.0|            131.04|
|           0.0|             150.0|
|           0.0|             260.0|
|           0.0|             500.0|
|           0.0|            4300.0|
|           0.0|             495.0|
|           0.0|             182.0|
|           0.0|             185.0|
|           0.0|            2817.0|
|           0.0|              54.6|
|           0.0|               1.0|
|           0.0|             100.0|
|           0.0|421.84999999999997|
|           0.0|             210.0|
|           0.0|            1010.0|
|           0.0|             200.0|
|           0.0|              20.0|
|           0.0|            4189.9|
|           0.0|            2912.0|
+--------------+------------------+
only showing top 20 rows



In [9]:
df_vuelos.count()

52234

In [10]:
df_vuelos_outlier.count()

21638

**Hallazgo relevante en calidad de datos**  

Se observa que cerca el 40% de los datos tiene este problema, es decir, donde la carga_bordo es mayor que la carga_ofrecida.  

Lo que hare para mejorar la calidad de estos datos sera hallar la carga ofrecida x cada vuelo y tipo de avion (revisando el numero de **vuelos** asociado a cada fila) y **tipo_equipo**(tipo de avion), en resumen hallare por cada tipo de avion el promedio de la carga que esta en capacidad de ofrecer de acuerdo a lo que se observe en otros registros donde los datos esten informados y se cumpla que la carga ofrecida > carga_bordo, y posterior a ello corregire todos aquellos registros (21638) donde la carga_ofrecida<carga_bordo.

In [11]:
df_vuelos = df_vuelos.withColumn("CargaOfrecidaxVuelo",df_vuelos["carga_ofrecida"]/df_vuelos["vuelos"])
carga_x_tipoAvion=df_vuelos.where((df_vuelos['carga_ofrecida']>0) & (df_vuelos["vuelos"]>0) & (df_vuelos['carga_ofrecida']>=df_vuelos['carga_bordo'])).groupBy('tipo_equipo').agg({'CargaOfrecidaxVuelo': 'mean'})
carga_x_tipoAvion.show()

+-----------+------------------------+
|tipo_equipo|avg(CargaOfrecidaxVuelo)|
+-----------+------------------------+
|       B762|      51306.192663794536|
|       F100|       2220.830546031652|
|        727|      24667.301313117314|
|       B742|                100000.0|
|       A332|       39366.66556450616|
|       B772|                101000.0|
|       JS32|      1459.7070665077326|
|        DH6|                  1375.0|
|       MD83|                  7700.0|
|       E190|       9380.331081043807|
|       E170|      2685.3028169014083|
|       A319|       7783.311898408734|
|       A320|      6202.2400507040375|
|        F10|      2958.3167421362027|
|       B743|                100000.0|
|       R722|      16484.352941176472|
|       AN26|       5616.670404984424|
|        M83|      6298.9361804711425|
|       AN32|                 7810.86|
|       A318|       6706.945432376806|
+-----------+------------------------+
only showing top 20 rows



Se crea una **UDF recalcula_carga_ofrecida** para recalcular la carga ofrecida con base en el promedio de carga ofrecida por tipo de avion y el numero de vuelos

In [12]:
df_vuelos = df_vuelos.join(carga_x_tipoAvion, how = 'left', on = 'tipo_equipo')
df_vuelos.show()

df_vuelos = df_vuelos.fillna({'avg(CargaOfrecidaxVuelo)': 0})

def recalcula_carga_ofrecida(carga_ofrecida_actual, carga_ofrecida_new, carga_bordo, numvuelos):
    if((carga_ofrecida_actual==0 or carga_ofrecida_actual<carga_bordo) and numvuelos>0 and carga_ofrecida_new>0):
        return carga_ofrecida_new*numvuelos
    else:
        return carga_ofrecida_actual

norm_udf = udf(lambda x,y,z,n: recalcula_carga_ofrecida(x, y, z,n),FloatType())

df_vuelos = df_vuelos.withColumn('carga_ofrecida_recalculada', norm_udf(df_vuelos['carga_ofrecida'],df_vuelos['avg(CargaOfrecidaxVuelo)'],df_vuelos['carga_bordo'],df_vuelos['vuelos']))
df_vuelos

+-----------+----+---+------+-------+----------+-------+--------------------+------+-------+--------------+---------+-----------+-------+-------------------+------------------------+
|tipo_equipo| ano|mes|origen|destino|tipo_vuelo|trafico|             empresa|vuelos| sillas|carga_ofrecida|pasajeros|carga_bordo|AnioMes|CargaOfrecidaxVuelo|avg(CargaOfrecidaxVuelo)|
+-----------+----+---+------+-------+----------+-------+--------------------+------+-------+--------------+---------+-----------+-------+-------------------+------------------------+
|       C560|2011| 10|   BOG|    CLO|         T|      N|     CENTRAL CHARTER|   4.0|    0.0|           0.0|      8.0|        0.0| 201110|                0.0|                    null|
|       BE40|2013|  4|   PUU|    BOG|         T|      N|         SEARCA S.A.|   3.0|    0.0|           0.0|      5.0|        0.0| 201304|                0.0|                    null|
|       BE9L|2011|  8|   BOG|    BOG|         T|      N|             RIO SUR|   2.0| 

DataFrame[tipo_equipo: string, ano: int, mes: int, origen: string, destino: string, tipo_vuelo: string, trafico: string, empresa: string, vuelos: double, sillas: double, carga_ofrecida: double, pasajeros: double, carga_bordo: double, AnioMes: int, CargaOfrecidaxVuelo: double, avg(CargaOfrecidaxVuelo): double, carga_ofrecida_recalculada: float]

In [13]:
#df_vuelos_outlier2=df_vuelos.where(df_vuelos['carga_ofrecida2']<df_vuelos['carga_bordo'])
#df_vuelos_outlier2.select('_c0','carga_ofrecida','carga_ofrecida2','carga_bordo').show()
df_vuelos.columns

['tipo_equipo',
 'ano',
 'mes',
 'origen',
 'destino',
 'tipo_vuelo',
 'trafico',
 'empresa',
 'vuelos',
 'sillas',
 'carga_ofrecida',
 'pasajeros',
 'carga_bordo',
 'AnioMes',
 'CargaOfrecidaxVuelo',
 'avg(CargaOfrecidaxVuelo)',
 'carga_ofrecida_recalculada']

In [15]:
df_vuelos.dtypes

[('tipo_equipo', 'string'),
 ('ano', 'int'),
 ('mes', 'int'),
 ('origen', 'string'),
 ('destino', 'string'),
 ('tipo_vuelo', 'string'),
 ('trafico', 'string'),
 ('empresa', 'string'),
 ('vuelos', 'double'),
 ('sillas', 'double'),
 ('carga_ofrecida', 'double'),
 ('pasajeros', 'double'),
 ('carga_bordo', 'double'),
 ('AnioMes', 'int'),
 ('CargaOfrecidaxVuelo', 'double'),
 ('avg(CargaOfrecidaxVuelo)', 'double'),
 ('carga_ofrecida_recalculada', 'float')]

A continacion podemos observar registros donde no se tenia carga_ofrecida, es decir estaba en 0 ahora quedara con el promedio de carga_ofrecifa para este tipo de avion x numero de vuelos que se puede ver en la columna **carga_ofrecida_recalculada**

In [16]:
df_vuelos.where((df_vuelos['carga_ofrecida']<df_vuelos['carga_bordo']) & (df_vuelos['carga_ofrecida_recalculada']>0) & (df_vuelos['carga_ofrecida']==0)).toPandas()

,tipo_equipo,ano,mes,origen,destino,tipo_vuelo,trafico,empresa,vuelos,sillas,carga_ofrecida,pasajeros,carga_bordo,AnioMes,CargaOfrecidaxVuelo,avg(CargaOfrecidaxVuelo),carga_ofrecida_recalculada
0,B190,2013,2,BOG,AUC,T,N,SEARCA S.A.,4.0,0.0,0.0,20.0,180.0,201302,0.0,1874.000000,7496.000000
1,AN32,2013,6,VVC,BOG,T,N,AER CARIBE LIMITADA,1.0,0.0,0.0,0.0,500.0,201306,0.0,7810.860000,7810.859863
2,JS32,2010,10,BOG,EYP,T,N,SARPA S.A.S,13.0,0.0,0.0,95.0,4300.0,201010,0.0,1459.707067,18976.191406
3,L410,2012,8,IBE,CPL,T,N,SEARCA S.A.,3.0,0.0,0.0,29.0,210.0,201208,0.0,411.326316,1233.979004
4,AN32,2013,7,BOG,VGP,T,N,AER CARIBE LIMITADA,2.0,0.0,0.0,0.0,1010.0,201307,0.0,7810.860000,15621.719727
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7026,AN32,2012,6,VVC,BOG,T,N,AER CARIBE LIMITADA,2.0,0.0,0.0,0.0,7800.0,201206,0.0,7810.860000,15621.719727
7027,B190,2013,7,NVA,BOG,T,N,PETROLEUM,9.0,0.0,0.0,112.0,100.0,201307,0.0,1874.000000,16866.000000
7028,DC3,2013,7,VVC,FLA,T,N,ALLAS S.A.S,1.0,0.0,0.0,4.0,1600.0,201307,0.0,3044.678154,3044.678223
7029,B190,2013,10,BOG,EYP,T,N,SEARCA S.A.,28.0,0.0,0.0,419.0,4628.0,201310,0.0,1874.000000,52472.000000


A continucion se puede observar la diferencia entre el valor original de la **carga_ofrecida** en muchos casos con valores errados por ser menores a la **carga_bordo** y la **carga_ofrecida_recalculada** con valores inferidos a partir del tipo de avion y el numero de vuelos, que si bien no logra corregir el problema en 100% si logra corregirlo en un alto % de los casos, en una proxima entrega puede que se analize como mejorar esta iniciativa y si otras variables (sillas, pasajeros) pueden aplicar para recalcular o corregir los valores de manera similar.

In [17]:
df_vuelos.where((df_vuelos['carga_ofrecida']<df_vuelos['carga_bordo']) & (df_vuelos['carga_ofrecida_recalculada']>0) & (df_vuelos['carga_ofrecida']>0)).toPandas()

,tipo_equipo,ano,mes,origen,destino,tipo_vuelo,trafico,empresa,vuelos,sillas,carga_ofrecida,pasajeros,carga_bordo,AnioMes,CargaOfrecidaxVuelo,avg(CargaOfrecidaxVuelo),carga_ofrecida_recalculada
0,DC3,2011,6,EYP,PZA,R,N,AIR COLOMBIA,0.0,0.0,1500.0,0.0,10920.00,201106,NaN,3044.678154,1500.000000
1,DH4,2011,2,BOG,EYP,R,N,AIRES,75.0,5850.0,10000.0,4106.0,42770.00,201102,133.333333,739.267597,55445.070312
2,B737,2011,8,BOG,MTR,R,N,AIRES,64.0,9472.0,20000.0,6630.0,113935.64,201108,312.500000,4790.871322,306615.750000
3,AN26,2013,9,SVI,VVC,R,N,SELVA,2.0,27.0,5000.0,0.0,23827.44,201309,2500.000000,5616.670405,11233.340820
4,JS32,2010,10,UIB,COG,R,N,AEROLINEA DE ANTIOQUIA S.A.,1.0,0.0,74.5,10.0,382.20,201010,74.500000,1459.707067,1459.707031
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
221,DH8D,2010,8,BOG,EYP,R,N,AIRES,102.0,3774.0,31342.5,3436.0,60744.32,201008,307.279412,970.707559,99012.171875
222,B737,2010,12,BOG,CLO,R,N,C.V. CARGO,22.0,0.0,151800.0,0.0,868347.48,201012,6900.000000,4790.871322,105399.171875
223,B737,2012,3,EYP,BOG,R,N,AIRES,43.0,6364.0,2716.5,4243.0,16896.88,201203,63.174419,4790.871322,206007.468750
224,DC3,2012,12,EYP,AUC,R,N,AIR COLOMBIA,2.0,0.0,3000.0,0.0,14560.00,201212,1500.000000,3044.678154,6089.356445


**Consulta bases de datos disponibles**  

Se crea base de datos **infravisibleLabs**


In [18]:
import mysql.connector
mydb = mysql.connector.connect(
  host="localhost",
  user="root",
  passwd="",
)
mycursor = mydb.cursor()
databases = ("show databases")

#Creo base de datos infravisibleLabs
mycursor.execute("CREATE DATABASE infravisibleLabs5")


mycursor.execute(databases)
for i in mycursor:
     print(i)


('Application',)
('Sales',)
('WWImportersDWH',)
('WWImportersTransactional',)
('Warehouse',)
('information_schema',)
('infravisibleLabs',)
('infravisibleLabs5',)
('mysql',)
('performance_schema',)
('sys',)


**CREACION TABLAS DE DIMENSIONES Y DE HECHOS**

**DIM_FECHA**

In [19]:
dim_fecha=df_vuelos.selectExpr('AnioMes as idFecha','ano as anio', 'mes').distinct().sort('AnioMes')
dim_fecha.show()

+-------+----+---+
|idFecha|anio|mes|
+-------+----+---+
| 201001|2010|  1|
| 201002|2010|  2|
| 201003|2010|  3|
| 201004|2010|  4|
| 201005|2010|  5|
| 201006|2010|  6|
| 201007|2010|  7|
| 201008|2010|  8|
| 201009|2010|  9|
| 201010|2010| 10|
| 201011|2010| 11|
| 201012|2010| 12|
| 201101|2011|  1|
| 201102|2011|  2|
| 201103|2011|  3|
| 201104|2011|  4|
| 201105|2011|  5|
| 201106|2011|  6|
| 201107|2011|  7|
| 201108|2011|  8|
+-------+----+---+
only showing top 20 rows



**DIM_GEOGRAFIA**  

Dado que es de conocimiento que el codigo de municipio, tiene ya de por si información del departamento, y que no es un codigo que represente solo al municipio como tal, en otra palabras es un código compuesto que tiene informacion del departamento y el municipio. Por ende se usuara usara dicho código como llave de la nuevo dimension Geografia, para representar municipio y departamento.

In [20]:
dim_geografia=df_aerop.selectExpr('gcd_municipio as id_Geografia','municipio','departamento').distinct().sort('gcd_municipio')
dim_geografia.show()

+------------+--------------------+------------+
|id_Geografia|           municipio|departamento|
+------------+--------------------+------------+
|        5031|              Amalfi|   Antioquia|
|        5051|           Arboletes|   Antioquia|
|        5147|              Carepa|   Antioquia|
|        5150|            Carolina|   Antioquia|
|        5172|           Chigorodó|   Antioquia|
|        5604|            Remedios|   Antioquia|
|        5847|               Urrao|   Antioquia|
|        8573|     Puerto Colombia|   Atlántico|
|       11001|        Bogotá, D.C.|Bogotá, D.C.|
|       13001| Cartagena de Indias|     Bolívar|
|       13212|             Córdoba|     Bolívar|
|       13430|            Magangué|     Bolívar|
|       13468|              Mompós|     Bolívar|
|       13673|      Santa Catalina|     Bolívar|
|       15516|               Paipa|      Boyacá|
|       15572|       Puerto Boyacá|      Boyacá|
|       18001|           Florencia|     Caquetá|
|       18753|San Vi

**DIM_CATEGORIA**

In [21]:
dim_categoria=df_aerop.selectExpr('substr(categoria,1,1) as id_Categoria','categoria').distinct().sort('categoria')
dim_categoria.show()

+------------+-------------+
|id_Categoria|    categoria|
+------------+-------------+
|           A|    Aeródromo|
|           I|Internacional|
|           N|     Nacional|
|           R|     Regional|
+------------+-------------+



**DIM_EMPRESA**

In [77]:
from pyspark.sql.functions import *
from pyspark.sql.window import *

window = Window.orderBy(col('monotonically_increasing_id'))

dim_empresa=df_vuelos.selectExpr('empresa').distinct().sort('empresa')

w = Window.orderBy("empresa")
dim_empresa = dim_tipoEmpresa.withColumn("id_Empresa", row_number().over(w))

dim_empresa=dim_empresa.selectExpr('id_Empresa','empresa')
dim_empresa.show()


+----------+--------------------+
|id_Empresa|             empresa|
+----------+--------------------+
|         1| AER CARIBE LIMITADA|
|         2|          AERO APOYO|
|         3|AERO TAXI GUAYMAR...|
|         4|AEROCHARTER ANDIN...|
|         5|       AEROCOL S.A.S|
|         6|AEROEJECUTIVOS DE...|
|         7|AEROEXPRESO DEL P...|
|         8|           AEROGALAN|
|         9|AEROLINEA DE ANTI...|
|        10|AEROLINEAS ALAS D...|
|        11|AEROLINEAS LLANER...|
|        12|        AEROMEL LTDA|
|        13|         AEROMENEGUA|
|        14|       AEROREPUBLICA|
|        15|           AEROSUCRE|
|        16|         AERUPIA SAS|
|        17|        AIR COLOMBIA|
|        18| AIR SIGNATURE S.A.S|
|        19|               AIRES|
|        20|             ALIANSA|
+----------+--------------------+
only showing top 20 rows



**DIM_AEROPUERTO**

In [48]:
dim_aeropuerto=df_aerop.selectExpr('sigla as id_Aeropuerto','gcd_municipio as id_Geografia','substr(categoria,1,1) as id_Categoria',\
'nombre','longitud_pista','ancho_pista', 'pbmo', 'elevacion','latitud', 'longitud')
dim_aeropuerto.show()

+-------------+------------+------------+--------------------+--------------+-----------+-------+---------+-------+--------+
|id_Aeropuerto|id_Geografia|id_Categoria|              nombre|longitud_pista|ancho_pista|   pbmo|elevacion|latitud|longitud|
+-------------+------------+------------+--------------------+--------------+-----------+-------+---------+-------+--------+
|          9CL|       85325|           A|        LA VENTUROSA|         620.0|       15.0| 2000.0|    459.0| 5.2388|-71.3035|
|          9MN|       68575|           A|   EL PEDRAL BONANZA|         800.0|       10.0| 3000.0|    282.0| 7.2417|-73.8151|
|          7HC|       99524|           A|          EL CAFUCHE|         850.0|       18.0| 4000.0|    384.0| 5.3152|-70.0184|
|          9AB|       85125|           A|           GETSEMANI|         800.0|       14.0| 2000.0|    304.0| 6.1506|-70.0294|
|          9CG|       85250|           A|SAN JOSE DEL ARIPORO|         570.0|       15.0| 2000.0|    318.0| 6.0475|-70.0128|


In [ ]:
df_aerop.columns

**FACT_VUELOS**

In [61]:
fact_vuelos_detalle=df_vuelos.selectExpr('AnioMes as idFecha', 'origen as id_Aeropuerto_Origen', 'destino as id_Aeropuerto_Destino', 'sillas','pasajeros','carga_ofrecida_recalculada as carga_ofrecida','carga_bordo','sillas-pasajeros as sillas_desaprovechadas', 'carga_ofrecida_recalculada-carga_bordo as carga_desaprovechada','vuelos','empresa')
fact_vuelos_detalle.toPandas()

fact_vuelos_detalle = fact_vuelos_detalle.join(dim_empresa, how = 'inner', on='empresa')
fact_vuelos_detalle.toPandas()

,empresa,idFecha,id_Aeropuerto_Origen,id_Aeropuerto_Destino,sillas,pasajeros,carga_ofrecida,carga_bordo,sillas_desaprovechadas,carga_desaprovechada,vuelos,id_Empresa
0,HELICOL,201312,BOG,EYP,0.0,47.0,7496.0,0.0,-47.0,7496.0,4.0,38
1,HELICOL,201312,EYP,VVC,0.0,1.0,1874.0,0.0,-1.0,1874.0,1.0,38
2,HELICOL,201312,NVA,PUU,0.0,35.0,3748.0,658.0,-35.0,3090.0,2.0,38
3,HELICOL,201312,EYP,VVC,0.0,0.0,1874.0,0.0,0.0,1874.0,1.0,38
4,HELICOL,201312,EYP,BOG,0.0,90.0,13118.0,0.0,-90.0,13118.0,7.0,38
...,...,...,...,...,...,...,...,...,...,...,...,...
52229,AVIANCA,201203,VUP,BOG,200.0,153.0,11078.0,173.0,47.0,10905.0,2.0,26
52230,AVIANCA,201312,BOG,CLO,68.0,0.0,356.0,260.0,68.0,96.0,1.0,26
52231,AVIANCA,201310,BOG,MTR,800.0,714.0,28085.0,5241.0,86.0,22844.0,8.0,26
52232,AVIANCA,201308,CLO,CTG,3600.0,3002.0,168269.0,4671.0,598.0,163598.0,24.0,26


In [62]:
fact_vuelos_detalle.createOrReplaceTempView("fact_vuelos")
fact_vuelos=spark.sql("SELECT idFecha " +
      ",id_Aeropuerto_Origen " +
      ",id_Aeropuerto_Destino " +
      ",id_Empresa " +                
      ", sum(sillas) as sillas" +
      ", sum(pasajeros) as pasajeros " +
      ", sum(carga_ofrecida) as carga_ofrecida " +
      ", sum(carga_bordo) as carga_bordo " +
      ", sum(sillas_desaprovechadas) sillas_desaprovechadas " +
      ", sum(carga_desaprovechada) as carga_desaprovechada " +
      ", sum(vuelos) as vuelos " + 
  "FROM fact_vuelos " +
  "group by idFecha " +
      ",id_Aeropuerto_Origen" +
      ",id_Aeropuerto_Destino" +
      ",id_Empresa")
fact_vuelos.count()



31696

**Carga datos en base de datos DWH**

In [63]:
# Configuración servidor base de datos multidimensional
user_md = 'root'
psswd_md = ''
db_multidimensional_connection_string = 'jdbc:mysql://localhost:3306/infravisibleLabs5'

In [64]:
# Persiste la dimensión dim_fecha en la base de datos multidimensional
dim_fecha.select('*').write.format('jdbc') \
.mode('append') \
.option('url', db_multidimensional_connection_string) \
.option('dbtable', 'dim_fecha') \
.option('user', user_md) \
.option('password', psswd_md) \
.option('driver', 'org.mariadb.jdbc.Driver') \
.save()

**Validación de lectura de datos dimensión dim_fecha recien insertados en la base de datos:**

In [65]:
def load_dim_fecha(spark):
    """Carga, de la base de datos de infravisibleLabs, la tabla dim_fecha

    Parámetros:
    --------------
    spark: 
        Sesión de spark.
    """

    return spark.read.format('jdbc')\
    .option('url', db_multidimensional_connection_string) \
    .option('dbtable', '''(SELECT *
    FROM  dim_fecha) AS temp_dimfecha''') \
    .option('user', user_md) \
    .option('password', psswd_md) \
    .option('driver', 'org.mariadb.jdbc.Driver') \
    .load()

load_dim_fecha(spark).show()

+-------+----+---+
|idFecha|anio|mes|
+-------+----+---+
| 201001|2010|  1|
| 201002|2010|  2|
| 201003|2010|  3|
| 201004|2010|  4|
| 201005|2010|  5|
| 201006|2010|  6|
| 201007|2010|  7|
| 201008|2010|  8|
| 201009|2010|  9|
| 201010|2010| 10|
| 201011|2010| 11|
| 201012|2010| 12|
| 201101|2011|  1|
| 201102|2011|  2|
| 201103|2011|  3|
| 201104|2011|  4|
| 201105|2011|  5|
| 201106|2011|  6|
| 201107|2011|  7|
| 201108|2011|  8|
+-------+----+---+
only showing top 20 rows



In [66]:
# Persiste la dimensión dim_geografia en la base de datos multidimensional
dim_geografia.select('*').write.format('jdbc') \
.mode('append') \
.option('url', db_multidimensional_connection_string) \
.option('dbtable', 'dim_geografia') \
.option('user', user_md) \
.option('password', psswd_md) \
.option('driver', 'org.mariadb.jdbc.Driver') \
.save()

In [67]:
# Persiste la dimensión dim_categoria en la base de datos multidimensional
dim_categoria.select('*').write.format('jdbc') \
.mode('append') \
.option('url', db_multidimensional_connection_string) \
.option('dbtable', 'dim_categoria') \
.option('user', user_md) \
.option('password', psswd_md) \
.option('driver', 'org.mariadb.jdbc.Driver') \
.save()

In [79]:
# Persiste la dimensión dim_empresa en la base de datos multidimensional
dim_empresa.select('*').write.format('jdbc') \
.mode('overwrite') \
.option('url', db_multidimensional_connection_string) \
.option('dbtable', 'dim_empresa') \
.option('user', user_md) \
.option('password', psswd_md) \
.option('driver', 'org.mariadb.jdbc.Driver') \
.save()

In [69]:
# Persiste la dimensión dim_aeropuerto en la base de datos multidimensional
dim_aeropuerto.select('*').write.format('jdbc') \
.mode('append') \
.option('url', db_multidimensional_connection_string) \
.option('dbtable', 'dim_aeropuerto') \
.option('user', user_md) \
.option('password', psswd_md) \
.option('driver', 'org.mariadb.jdbc.Driver') \
.save()

In [72]:
# Persiste la tabla de hechos fact_vuelos en la base de datos multidimensional
fact_vuelos.select('*').write.format('jdbc') \
.mode('append') \
.option('url', db_multidimensional_connection_string) \
.option('dbtable', 'fact_vuelos') \
.option('user', user_md) \
.option('password', psswd_md) \
.option('driver', 'org.mariadb.jdbc.Driver') \
.save()

In [81]:
os.popen("mysqldump -h localhost -u "+user_md+" -p 'infravisibleLabs' > archivo_mysql_respaldo_S5.sql")

In [80]:
 def load_fact_vuelos(spark):
    """Carga, de la base de datos de infravisibleLabs, la tabla dim_fecha

    Parámetros:
    --------------
    spark: 
        Sesión de spark.
    """

    return spark.read.format('jdbc')\
    .option('url', db_multidimensional_connection_string) \
    .option('dbtable', '''(SELECT *
    FROM  fact_vuelos) AS temp_fact_vuelos''') \
    .option('user', user_md) \
    .option('password', psswd_md) \
    .option('driver', 'org.mariadb.jdbc.Driver') \
    .load()

load_fact_vuelos(spark).count()

71

In [74]:
fact_vuelos.count()

31696

**INICIO TRABAJO EN GRUPO - COBERTURAS**

In [55]:
import pandas 
PATH = "./data/"

df_aerop = spark.read.load(PATH+"aeropuertos_grupo.csv",format="csv", sep=";", inferSchema="true", header="true")

df_centros = spark.read.load(PATH+"Cobertura Aerea de Centros poblados por categoria de AereopuertoConCodigoCentroPoblado.csv",format="csv", sep=";", inferSchema="true", header="true")

df_centros.toPandas()

#renombro columnas con espacios

for col in df_centros.columns:
    df_centros = df_centros.withColumnRenamed(col,col.replace(" ", "_"))


df_centros.dtypes



[('Codigo_Centro_Poblado', 'int'),
 ('Nombre_centro_poblado', 'string'),
 ('Aeropuerto', 'string'),
 ('Distancia(Km)', 'string'),
 ('Cobertura', 'boolean'),
 ('Aerodromo', 'string'),
 ('D_Aerodromo', 'string'),
 ('Regional', 'string'),
 ('D_Regional', 'string'),
 ('Nacional', 'string'),
 ('D_Nacional', 'string'),
 ('Internacional', 'string'),
 ('D_Internacional', 'string'),
 ('Tipo_Cobertura', 'string')]

In [41]:
df1=df_centros.selectExpr('Codigo_Centro_Poblado as id_centro_poblado','Aerodromo as sigla', '"A" as id_categoria', 'substr(Tipo_Cobertura,1,1) as id_tipo_cobertura','D_Aerodromo as distancia')
df2=df_centros.selectExpr('Codigo_Centro_Poblado as id_centro_poblado','Regional as sigla', '"R" as id_categoria', 'substr(Tipo_Cobertura,1,1) as id_tipo_cobertura','D_Regional as distancia')
df3=df_centros.selectExpr('Codigo_Centro_Poblado as id_centro_poblado','Nacional as sigla', '"N" as id_categoria', 'substr(Tipo_Cobertura,1,1) as id_tipo_cobertura','D_Nacional as distancia')
df4=df_centros.selectExpr('Codigo_Centro_Poblado as id_centro_poblado','Internacional as sigla', '"I" as id_categoria', 'substr(Tipo_Cobertura,1,1) as id_tipo_cobertura','D_Internacional as distancia')

final_df=df1.union(df2)\
         .union(df3)\
         .union(df4)
print(df1.count())
print(df2.count())
print(df3.count())
print(df4.count())
print(final_df.count())

df_centros2=final_df.sort('id_centro_poblado')
#df_centros2.where('id_centro_poblado in(99773000,99773001)').show()
df_centros2.show()

7879
7879
7879
7879
31516
+-----------------+-----+------------+-----------------+-----------+
|id_centro_poblado|sigla|id_categoria|id_tipo_cobertura|  distancia|
+-----------------+-----+------------+-----------------+-----------+
|          5001000|  MDE|           I|                I|19,78362876|
|          5001000|  7IC|           A|                I|52,67179641|
|          5001000|  EOH|           N|                I| 3,07967291|
|          5001000|  URR|           R|                I|62,45094172|
|          5001001|  MDE|           I|                I|35,59064494|
|          5001001|  7FX|           A|                I|61,18342718|
|          5001001|  URR|           R|                I|49,85921309|
|          5001001|  EOH|           N|                I|17,55454828|
|          5001004|  URR|           R|                I|71,82711814|
|          5001004|  EOH|           N|                I|9,853178054|
|          5001004|  7IC|           A|                I|49,77671062|
|       

In [56]:
from pyspark.sql.functions import *
from pyspark.sql.window import *


window = Window.orderBy(col('monotonically_increasing_id'))

dim_categoria_aeropuerto=df_centros.selectExpr('substr(Tipo_Cobertura,1,1) as id_categoria','Tipo_Cobertura as categoria').distinct().sort('Tipo_Cobertura')
dim_categoria_aeropuerto.show()

'''
w = Window.orderBy("categoria")
dim_categoria = dim_categoria.withColumn("id_categoria", row_number().over(w))

dim_categoria=dim_categoria.selectExpr('id_categoria','categoria')
dim_categoria.show()
'''

+------------+-------------+
|id_categoria|    categoria|
+------------+-------------+
|           A|    Aer�dromo|
|           I|Internacional|
|           N|     Nacional|
|           R|     Regional|
|           S|Sin cobertura|
+------------+-------------+



'\nw = Window.orderBy("categoria")\ndim_categoria = dim_categoria.withColumn("id_categoria", row_number().over(w))\n\ndim_categoria=dim_categoria.selectExpr(\'id_categoria\',\'categoria\')\ndim_categoria.show()\n'

In [53]:

df_idaerop=df_aerop.selectExpr('id as id_aeropuerto','sigla').where('vigente="S"')

hechos_cobertura_centros_poblados= df_centros2.join(df_idaerop, how = 'inner', on='sigla')

hechos_cobertura_centros_poblados = hechos_cobertura_centros_poblados.selectExpr('4702 as id_fecha', 'id_centro_poblado','id_aeropuerto','id_categoria','id_tipo_cobertura','distancia').sort('id_centro_poblado')

hechos_cobertura_centros_poblados.show()

hechos_cobertura_centros_poblados.dtypes

+--------+-----------------+-------------+------------+-----------------+-----------+
|id_fecha|id_centro_poblado|id_aeropuerto|id_categoria|id_tipo_cobertura|  distancia|
+--------+-----------------+-------------+------------+-----------------+-----------+
|    4702|          5001000|          156|           A|                I|52,67179641|
|    4702|          5001000|          833|           R|                I|62,45094172|
|    4702|          5001001|           11|           A|                I|61,18342718|
|    4702|          5001001|          833|           R|                I|49,85921309|
|    4702|          5001004|          833|           R|                I|71,82711814|
|    4702|          5001004|          156|           A|                I|49,77671062|
|    4702|          5001009|          156|           A|                I|50,07859843|
|    4702|          5001009|          833|           R|                I|57,80080211|
|    4702|          5001010|          156|           A

[('id_fecha', 'int'),
 ('id_centro_poblado', 'int'),
 ('id_aeropuerto', 'int'),
 ('id_categoria', 'string'),
 ('id_tipo_cobertura', 'string'),
 ('distancia', 'string')]

In [57]:
df_geocentro= spark.read.load(PATH+"Codigos_manifiesto_de_carga.csv",format="csv", sep=";", inferSchema="true", header="true")
df_geocentro=df_geocentro.selectExpr('CODIGO as id_centro_poblado','MUNICIPIO as municipio','DEPARTAMENTO as departamento')

dim_centro_poblado=df_centros.selectExpr('Codigo_Centro_Poblado as id_centro_poblado','nombre_centro_poblado as nombre_centro_poblado')
dim_centro_poblado = dim_centro_poblado.join(df_geocentro, how = 'inner', on='id_centro_poblado')
dim_centro_poblado.show()

+-----------------+---------------------+----------+------------+
|id_centro_poblado|nombre_centro_poblado| municipio|departamento|
+-----------------+---------------------+----------+------------+
|          5001000|             MEDELL�N|  MEDELLIN|   Antioquia|
|          5001001|             PALMITAS|  MEDELLIN|   Antioquia|
|          5001004|          SANTA ELENA|  MEDELLIN|   Antioquia|
|          5001009|            ALTAVISTA|  MEDELLIN|   Antioquia|
|          5001010|          AGUAS FR�AS|  MEDELLIN|   Antioquia|
|          5002000|            ABEJORRAL| ABEJORRAL|   Antioquia|
|          5002002|           PANTANILLO| ABEJORRAL|   Antioquia|
|          5004000|             ABRIAQU�|  ABRIAQUI|   Antioquia|
|          5021000|           ALEJANDR�A|ALEJANDRIA|   Antioquia|
|          5030000|                AMAG�|     AMAGA|   Antioquia|
|          5030001|             CAMILO C|     AMAGA|   Antioquia|
|          5030003|           LA CLARITA|     AMAGA|   Antioquia|
|         